# Q5: NECO — OOD Detection via Neural Collapse

This notebook implements the **NECO score** for OOD detection and benchmarks it against baselines.

The NECO score (eq. 6 of the paper) exploits NC1 + NC2 + NC5 to separate ID from OOD:

$$\text{NECO}(\mathbf{x}) = \frac{\|P \, h_\omega(\mathbf{x})\|}{\|h_\omega(\mathbf{x})\|}$$

where $P$ projects onto the top-$d$ principal components of ID features.

We study:
1. **NECO score** computation on SVHN and CIFAR-10 as OOD datasets
2. **PCA dimension sweep** — finding the optimal ETF approximation
3. **Score distributions** — histograms & ROC curves
4. **Comparison with baselines** — MSP, MaxLogit, Energy
5. **PCA 2D visualization** — how ID clusters separate from OOD

Reference:
> Ben Ammar et al., *"NECO: Neural Collapse Based Out-of-Distribution Detection"*, ICLR 2024.

## Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository
import os
if not os.path.exists('/content/OOD-Detection-Project---CSC_5IA23'):
    !git clone https://github.com/DiegoFleury/OOD-Detection-Project---CSC_5IA23/

!git stash
!git checkout contente
!git pull
%cd /content/OOD-Detection-Project---CSC_5IA23

In [ ]:
# Install dependencies
!pip install -q torch torchvision matplotlib seaborn scikit-learn pyyaml imageio tqdm

In [ ]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
import glob
import re
import os

from src.models import ResNet18
from src.data import get_cifar100_loaders

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load config
with open('configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration:")
print(yaml.dump(config, default_flow_style=False))

## 1. Load Data

In [ ]:
# ID data: CIFAR-100
print("Loading CIFAR-100 (ID) dataset...")

train_loader, val_loader, test_loader = get_cifar100_loaders(
    data_dir=config['data']['data_dir'],
    batch_size=config['training']['batch_size'],
    num_workers=config['data']['num_workers'],
    augment=False,
    val_split=config['training']['val_split']
)

print(f"Train batches: {len(train_loader)}")
print(f"Test batches:  {len(test_loader)}")

In [ ]:
# OOD data: SVHN + CIFAR-10
import torchvision
import torchvision.transforms as transforms

ood_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    ),
])

# SVHN
print("Loading SVHN (OOD)...")
svhn_dataset = torchvision.datasets.SVHN(
    root=config['data']['data_dir'], split='test',
    transform=ood_transform, download=True,
)
svhn_loader = torch.utils.data.DataLoader(
    svhn_dataset, batch_size=config['training']['batch_size'],
    shuffle=False, num_workers=config['data']['num_workers'],
)
print(f"SVHN test samples: {len(svhn_dataset)}")

# CIFAR-10
print("Loading CIFAR-10 (OOD)...")
cifar10_dataset = torchvision.datasets.CIFAR10(
    root=config['data']['data_dir'], train=False,
    transform=ood_transform, download=True,
)
cifar10_loader = torch.utils.data.DataLoader(
    cifar10_dataset, batch_size=config['training']['batch_size'],
    shuffle=False, num_workers=config['data']['num_workers'],
)
print(f"CIFAR-10 test samples: {len(cifar10_dataset)}")

## 2. Load Trained Model

In [ ]:
print("Creating ResNet-18 model...")
model = ResNet18(num_classes=config['model']['num_classes'])

# Load best checkpoint
checkpoint_dir = config['paths']['checkpoints']
checkpoints = glob.glob(os.path.join(checkpoint_dir, 'resnet18_cifar100_*.pth'))

def get_epoch_num(path):
    match = re.search(r'epoch(\d+)', path)
    return int(match.group(1)) if match else 0

latest = max(checkpoints, key=get_epoch_num)
epoch_num = get_epoch_num(latest)

ckpt = torch.load(latest, map_location=device, weights_only=False)
if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
    model.load_state_dict(ckpt['model_state_dict'])
elif isinstance(ckpt, dict) and 'state_dict' in ckpt:
    model.load_state_dict(ckpt['state_dict'])
else:
    model.load_state_dict(ckpt)

model = model.to(device)
model.eval()

print(f"✅ Loaded: {os.path.basename(latest)} (epoch {epoch_num})")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 3. Import NECO Module

In [ ]:
from src.neural_collapse.neco import (
    compute_neco_scores,
    compute_baseline_scores,
    evaluate_ood_detection,
    plot_neco_distributions,
    plot_neco_pca_2d,
    plot_pca_dim_sweep,
    NECOResult,
)

print("✅ NECO module imported successfully!")

In [ ]:
# Output directories
figures_dir = os.path.join(config['paths']['figures'], 'neco')
metrics_dir = config['paths']['metrics']
os.makedirs(figures_dir, exist_ok=True)
os.makedirs(metrics_dir, exist_ok=True)

## 4. NECO Score — OOD Detection

The NECO score projects features onto the top-$(C-1)$ principal components fitted on ID training data.

**Intuition:** If NC1+NC2 hold, ID features live in a $(C-1)$-dimensional ETF subspace. NC5 pushes OOD features orthogonal to this subspace, so their projection norm is smaller.

In [ ]:
# NECO score — SVHN as OOD
print("🔬 Computing NECO scores (OOD = SVHN)...")

result_svhn = compute_neco_scores(
    model=model,
    id_loader=test_loader,
    ood_loader=svhn_loader,
    device=device,
    num_classes=config['model']['num_classes'],
    id_train_loader=train_loader,
    pca_dim=config['model']['num_classes'] - 1,   # C-1 = 99
    use_maxlogit=False,
)

print(f"\n📊 CIFAR-100 vs SVHN:")
print(f"   PCA dim:  {result_svhn.pca_dim}")
print(f"   AUROC:    {result_svhn.auroc:.4f} ({result_svhn.auroc*100:.2f}%)")
print(f"   FPR95:    {result_svhn.fpr95:.4f} ({result_svhn.fpr95*100:.2f}%)")

In [ ]:
# NECO score — CIFAR-10 as OOD
print("🔬 Computing NECO scores (OOD = CIFAR-10)...")

result_cifar10 = compute_neco_scores(
    model=model,
    id_loader=test_loader,
    ood_loader=cifar10_loader,
    device=device,
    num_classes=config['model']['num_classes'],
    id_train_loader=train_loader,
    pca_dim=config['model']['num_classes'] - 1,
    use_maxlogit=False,
)

print(f"\n📊 CIFAR-100 vs CIFAR-10:")
print(f"   PCA dim:  {result_cifar10.pca_dim}")
print(f"   AUROC:    {result_cifar10.auroc:.4f} ({result_cifar10.auroc*100:.2f}%)")
print(f"   FPR95:    {result_cifar10.fpr95:.4f} ({result_cifar10.fpr95*100:.2f}%)")

### 4.1 NECO Score Distributions

Reproducing Figures E.16 and E.17 from the paper.

In [ ]:
fig_dist_svhn = plot_neco_distributions(
    result_svhn, id_name="CIFAR-100", ood_name="SVHN",
    save_dir=figures_dir,
)
plt.show()

In [ ]:
fig_dist_c10 = plot_neco_distributions(
    result_cifar10, id_name="CIFAR-100", ood_name="CIFAR-10",
    save_dir=figures_dir,
)
plt.show()

## 5. PCA Dimension Sweep

Following Section C.5 and Table C.5: we vary the PCA dimension $d$ to find the optimal ETF subspace approximation.

In [ ]:
# Sweep PCA dimensions
pca_dims = [10, 20, 30, 50, 70, 99, 130, 150, 200, 250, 300, 400, 500]
pca_dims = [d for d in pca_dims if d < 510]  # ResNet-18 feature dim = 512

sweep_svhn = {}
sweep_cifar10 = {}

print("Sweeping PCA dimensions...")
for d in pca_dims:
    sweep_svhn[d] = compute_neco_scores(
        model=model, id_loader=test_loader, ood_loader=svhn_loader,
        device=device, num_classes=config['model']['num_classes'],
        id_train_loader=train_loader, pca_dim=d,
    )
    sweep_cifar10[d] = compute_neco_scores(
        model=model, id_loader=test_loader, ood_loader=cifar10_loader,
        device=device, num_classes=config['model']['num_classes'],
        id_train_loader=train_loader, pca_dim=d,
    )

    print(f"  d={d:>3d} | SVHN: AUROC={sweep_svhn[d].auroc:.4f} "
          f"FPR95={sweep_svhn[d].fpr95:.4f} | "
          f"CIFAR-10: AUROC={sweep_cifar10[d].auroc:.4f} "
          f"FPR95={sweep_cifar10[d].fpr95:.4f}")

In [ ]:
# Plot sweep — SVHN
fig_sweep_svhn = plot_pca_dim_sweep(
    sweep_svhn, ood_name="SVHN", save_dir=figures_dir,
)
plt.show()

# Plot sweep — CIFAR-10
fig_sweep_c10 = plot_pca_dim_sweep(
    sweep_cifar10, ood_name="CIFAR-10", save_dir=figures_dir,
)
plt.show()

# Best dimensions
for name, sweep in [('SVHN', sweep_svhn), ('CIFAR-10', sweep_cifar10)]:
    best_d = min(sweep.keys(), key=lambda d: sweep[d].fpr95)
    print(f"Best d for {name}: {best_d} "
          f"(AUROC={sweep[best_d].auroc:.4f}, FPR95={sweep[best_d].fpr95:.4f})")

In [ ]:
# Combined PCA sweep plot (both OOD datasets)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
dims = sorted(sweep_svhn.keys())

ax = axes[0]
ax.plot(dims, [sweep_svhn[d].auroc * 100 for d in dims],
        'b-o', markersize=4, label='SVHN')
ax.plot(dims, [sweep_cifar10[d].auroc * 100 for d in dims],
        'g-s', markersize=4, label='CIFAR-10')
ax.set_xlabel('PCA dimension (d)', fontsize=12)
ax.set_ylabel('AUROC (%)', fontsize=12)
ax.set_title('AUROC vs PCA Dimension', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(dims, [sweep_svhn[d].fpr95 * 100 for d in dims],
        'b-o', markersize=4, label='SVHN')
ax.plot(dims, [sweep_cifar10[d].fpr95 * 100 for d in dims],
        'g-s', markersize=4, label='CIFAR-10')
ax.set_xlabel('PCA dimension (d)', fontsize=12)
ax.set_ylabel('FPR95 (%)', fontsize=12)
ax.set_title('FPR95 vs PCA Dimension', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

fig.suptitle('NECO: Effect of PCA Dimension (ResNet-18 / CIFAR-100)', fontsize=15, y=1.02)
fig.tight_layout()
plt.savefig(os.path.join(figures_dir, 'neco_pca_dim_sweep_combined.png'),
            dpi=150, bbox_inches='tight')
plt.show()

## 6. PCA 2D Feature Projections

Reproducing Figure 2 / D.14: ID classes form distinct clusters on the ETF subspace, while OOD data projects near the origin.

In [ ]:
fig_pca_svhn = plot_neco_pca_2d(
    model=model, id_loader=test_loader, ood_loader=svhn_loader,
    device=device, num_classes=config['model']['num_classes'],
    id_name="CIFAR-100", ood_name="SVHN",
    max_samples=3000, save_dir=figures_dir,
)
plt.show()

In [ ]:
fig_pca_c10 = plot_neco_pca_2d(
    model=model, id_loader=test_loader, ood_loader=cifar10_loader,
    device=device, num_classes=config['model']['num_classes'],
    id_name="CIFAR-100", ood_name="CIFAR-10",
    max_samples=3000, save_dir=figures_dir,
)
plt.show()

## 7. Comparison with Baselines

We compare NECO against:
- **MSP** — Maximum Softmax Probability (Hendrycks & Gimpel, 2017)
- **MaxLogit** — Max Logit (Hendrycks et al., 2022)
- **Energy** — Energy score = logsumexp(logits) (Liu et al., 2020)

In [ ]:
# Compute baselines
baselines = {}

for method in ['msp', 'maxlogit', 'energy']:
    for ood_name, ood_ldr in [('SVHN', svhn_loader), ('CIFAR-10', cifar10_loader)]:
        id_scores = compute_baseline_scores(model, test_loader, device, method)
        ood_scores = compute_baseline_scores(model, ood_ldr, device, method)
        metrics = evaluate_ood_detection(id_scores, ood_scores)
        baselines[(method, ood_name)] = metrics
        print(f"{method:>10s} | {ood_name:>10s} | "
              f"AUROC={metrics['auroc']:.4f} FPR95={metrics['fpr95']:.4f}")

In [ ]:
# Summary table
print("\n" + "=" * 70)
print("OOD DETECTION RESULTS — ResNet-18 / CIFAR-100")
print("=" * 70)
print(f"{'Method':>12s} | {'OOD Dataset':>10s} | {'AUROC (%)':>10s} | {'FPR95 (%)':>10s}")
print("-" * 70)

method_labels = {'msp': 'Softmax', 'maxlogit': 'MaxLogit', 'energy': 'Energy'}

for method in ['msp', 'maxlogit', 'energy']:
    for ood_name in ['SVHN', 'CIFAR-10']:
        m = baselines[(method, ood_name)]
        print(f"{method_labels[method]:>12s} | {ood_name:>10s} | "
              f"{m['auroc']*100:>10.2f} | {m['fpr95']*100:>10.2f}")

print(f"{'NECO':>12s} | {'SVHN':>10s} | "
      f"{result_svhn.auroc*100:>10.2f} | {result_svhn.fpr95*100:>10.2f}")
print(f"{'NECO':>12s} | {'CIFAR-10':>10s} | "
      f"{result_cifar10.auroc*100:>10.2f} | {result_cifar10.fpr95*100:>10.2f}")
print("=" * 70)

# Averages
print("\nAverages across OOD datasets:")
for method in ['msp', 'maxlogit', 'energy']:
    avg_auroc = np.mean([baselines[(method, o)]['auroc'] for o in ['SVHN', 'CIFAR-10']])
    avg_fpr = np.mean([baselines[(method, o)]['fpr95'] for o in ['SVHN', 'CIFAR-10']])
    print(f"  {method_labels[method]:>10s}: AUROC={avg_auroc*100:.2f}%  FPR95={avg_fpr*100:.2f}%")

neco_avg_auroc = np.mean([result_svhn.auroc, result_cifar10.auroc])
neco_avg_fpr = np.mean([result_svhn.fpr95, result_cifar10.fpr95])
print(f"  {'NECO':>10s}: AUROC={neco_avg_auroc*100:.2f}%  FPR95={neco_avg_fpr*100:.2f}%")

## 8. Save Results

In [ ]:
import json

neco_results = {
    'model': 'ResNet-18',
    'id_dataset': 'CIFAR-100',
    'checkpoint_epoch': epoch_num,
    'neco': {
        'SVHN': {
            'auroc': result_svhn.auroc,
            'fpr95': result_svhn.fpr95,
            'pca_dim': result_svhn.pca_dim,
        },
        'CIFAR-10': {
            'auroc': result_cifar10.auroc,
            'fpr95': result_cifar10.fpr95,
            'pca_dim': result_cifar10.pca_dim,
        },
    },
    'baselines': {
        method: {
            ood: baselines[(method, ood)]
            for ood in ['SVHN', 'CIFAR-10']
        }
        for method in ['msp', 'maxlogit', 'energy']
    },
    'pca_sweep': {
        'SVHN': {d: {'auroc': r.auroc, 'fpr95': r.fpr95} for d, r in sweep_svhn.items()},
        'CIFAR-10': {d: {'auroc': r.auroc, 'fpr95': r.fpr95} for d, r in sweep_cifar10.items()},
    },
}

json_path = os.path.join(metrics_dir, 'neco_results.json')
with open(json_path, 'w') as f:
    json.dump(neco_results, f, indent=2)
print(f"💾 Saved: {json_path}")

## 9. Final Summary

In [ ]:
print("\n" + "=" * 60)
print("NECO OOD DETECTION SUMMARY")
print("=" * 60)

print(f"\nModel: ResNet-18 / CIFAR-100 (epoch {epoch_num})")
print(f"PCA dim: {result_svhn.pca_dim} (C-1 = {config['model']['num_classes']-1})")

print(f"\n--- NECO Results ---")
print(f"  SVHN:     AUROC = {result_svhn.auroc*100:.2f}%  |  FPR95 = {result_svhn.fpr95*100:.2f}%")
print(f"  CIFAR-10: AUROC = {result_cifar10.auroc*100:.2f}%  |  FPR95 = {result_cifar10.fpr95*100:.2f}%")
print(f"  Average:  AUROC = {neco_avg_auroc*100:.2f}%  |  FPR95 = {neco_avg_fpr*100:.2f}%")

print(f"\n--- Best baseline ---")
best_method = min(['msp', 'maxlogit', 'energy'],
    key=lambda m: np.mean([baselines[(m, o)]['fpr95'] for o in ['SVHN', 'CIFAR-10']]))
best_avg_fpr = np.mean([baselines[(best_method, o)]['fpr95'] for o in ['SVHN', 'CIFAR-10']])
best_avg_auroc = np.mean([baselines[(best_method, o)]['auroc'] for o in ['SVHN', 'CIFAR-10']])
print(f"  {method_labels[best_method]}: AUROC={best_avg_auroc*100:.2f}% FPR95={best_avg_fpr*100:.2f}%")

if neco_avg_fpr < best_avg_fpr:
    improvement = (best_avg_fpr - neco_avg_fpr) / best_avg_fpr * 100
    print(f"  → NECO improves FPR95 by {improvement:.1f}% relative")

print(f"\n--- Files Saved ---")
print(f"  Figures:  {figures_dir}/")
print(f"  Metrics:  {json_path}")

print("\n" + "=" * 60)

## 10. Commit Results to GitHub

In [ ]:
# !git add results/figures/neco/
# !git add results/metrics/neco_results.json
# !git commit -m "Add Q5 NECO OOD detection: scores, baselines, PCA sweep"
# !git push
#
# print("Results committed to GitHub!")